<a href="https://colab.research.google.com/github/elangbijak4/Universal-Language-and-Protocol-Construction/blob/main/Demo_2_Contoh_Konstruksi_Protokol_Alien_berdasarkan_Bahasa_Universal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --------------------------
# PFUL Protocol Demo (All-in-one)
# Paste/run this entire block in Google Colab
# --------------------------

import json
from math import isqrt

# --------------------------
# Utilities: primes, factorize, fibonacci (small helper)
# --------------------------
def sieve_primes(n):
    if n < 2:
        return []
    sieve = [True] * (n+1)
    sieve[0:2] = [False, False]
    for p in range(2, isqrt(n) + 1):
        if sieve[p]:
            step = p*p
            sieve[step:n+1:p] = [False] * (((n - step)//p) + 1)
    return [i for i, isprime in enumerate(sieve) if isprime]

def factorize(n, primes_list=None):
    if n < 2:
        return {}
    if primes_list is None:
        primes_list = sieve_primes(int(max(100, isqrt(n)+10)))
    rem = n
    factors = {}
    for p in primes_list:
        if p*p > rem:
            break
        if rem % p == 0:
            cnt = 0
            while rem % p == 0:
                rem //= p
                cnt += 1
            factors[p] = cnt
    if rem > 1:
        factors[rem] = factors.get(rem, 0) + 1
    return factors

# small primes base for demo
PRIMES = sieve_primes(2000)

# --------------------------
# PTL Grammar construction (Sender side)
# --------------------------
# We create a tiny alphabet and state set and map each symbol/state/direction to a unique prime.
alphabet = ['0','1','B']  # B = blank
states = ['q0','qH']      # q0 = running, qH = halting
dirs = ['L','R']

# assign primes from PRIMES (choose non-overlapping region for clarity)
# (for real-world: we'd choose a canonical mapping and include it in handshake)
symbol2prime = {s: PRIMES[i] for i,s in enumerate(alphabet)}              # 2,3,5 ...
state2prime  = {q: PRIMES[50 + i] for i,q in enumerate(states)}          # offset so states have different primes
dir2prime    = {d: PRIMES[100 + i] for i,d in enumerate(dirs)}          # directions at another offset

print("Sender: symbol2prime:", symbol2prime)
print("Sender: state2prime: ", state2prime)
print("Sender: dir2prime:   ", dir2prime)

# Define transitions for the bit-flipper TM:
# semantics: (q0,'0') -> (q0,'1',R)
#            (q0,'1') -> (q0,'0',R)
#            (q0,'B') -> (qH,'B',R)  # on blank, halt (go to halting state)
transitions_tuples = [
    ('q0','0','q0','1','R'),
    ('q0','1','q0','0','R'),
    ('q0','B','qH','B','R'),
]

# Encode each transition as product of primes:
# T = p_q * p_a * p_q' * p_b * p_D
def encode_transition(tup, s2p=symbol2prime, q2p=state2prime, d2p=dir2prime):
    q,a,qn,b,D = tup
    return q2p[q] * s2p[a] * q2p[qn] * s2p[b] * d2p[D]

encoded_transitions = [encode_transition(t) for t in transitions_tuples]
print("\nSender: encoded transitions (integers):")
for t_enc in encoded_transitions:
    print(t_enc)

# Build a handshake object containing the blueprint grammar
handshake = {
    'symbol2prime': symbol2prime,
    'state2prime': state2prime,
    'dir2prime': dir2prime,
    'transitions': encoded_transitions
}

# Simulate sending handshake as JSON (in real protocol: transmit this over channel)
handshake_json = json.dumps(handshake)
print("\n--- Handshake JSON (to be sent) ---")
print(handshake_json)

# --------------------------
# Sender: build a payload using the same PTL encoding style
# We'll encode an initial tape like: 0 1 1 0 B (B indicates end) with head at position 0 and state q0.
# For simplicity the payload will be (state_prime, tape_dict), but we will serialize in prime/config-style integer too.
# --------------------------
# encode configuration as integer: conf = p_state * product_{pos}( symbol_prime ** (pos+2) )
def encode_configuration(state, tape_dict, q2p=state2prime, s2p=symbol2prime):
    conf = q2p[state]
    for pos,sym in tape_dict.items():
        conf *= (s2p[sym] ** (pos + 2))
    return conf

# start tape: positions 0..4, with blank at last position
tape = {0:'0', 1:'1', 2:'1', 3:'0', 4:'B'}
initial_state = 'q0'
conf_integer = encode_configuration(initial_state, tape)
print("\nSender: initial configuration integer:", conf_integer)
print(" (this integer encodes state + tape and will be sent as payload)")

# Build the full "message" bundle: handshake + payload
message_bundle = {
    'handshake': handshake,
    'payload_conf': conf_integer
}

# Simulate sending: serialize to JSON (payload_conf is integer so JSON OK)
message_json = json.dumps(message_bundle)
print("\n--- Message bundle (handshake + payload) ready to send --- (length chars):", len(message_json))

# --------------------------
# RECEIVER SIDE: gets message_json, reconstructs grammar, rebuilds TM, decodes payload and simulates TM
# --------------------------

print("\n\n=== RECEIVER: receive JSON and reconstruct grammar & machine ===")

recv = json.loads(message_json)  # in real life, would be received from comms
recv_handshake = recv['handshake']
# Reconstruct mappings
r_symbol2prime = {k:int(v) for k,v in recv_handshake['symbol2prime'].items()}
r_state2prime  = {k:int(v) for k,v in recv_handshake['state2prime'].items()}
r_dir2prime    = {k:int(v) for k,v in recv_handshake['dir2prime'].items()}
r_trans_enc    = [int(x) for x in recv_handshake['transitions']]

# Build reverse maps (primes -> symbol/state/dir)
r_prime2symbol = {v:k for k,v in r_symbol2prime.items()}
r_prime2state  = {v:k for k,v in r_state2prime.items()}
r_prime2dir    = {v:k for k,v in r_dir2prime.items()}

print("Receiver rebuilt symbol2prime:", r_symbol2prime)
print("Receiver rebuilt state2prime: ", r_state2prime)
print("Receiver rebuilt dir2prime:   ", r_dir2prime)

# Decode each encoded transition integer by prime factorization,
# then map factors back to (q,a,q',b,D) using the reverse maps.
def decode_transition_integer(T, prime_bases=PRIMES):
    fac = factorize(T, prime_bases)
    # We expect five primes in the factorization corresponding to q, a, q', b, D
    # Because primes chosen disjointly (states in different offset region), we can detect role by which reverse map contains it.
    roles = {'q':None,'a':None,'q2':None,'b':None,'D':None}
    for p in fac:
        if p in r_prime2state and roles['q'] is None:
            roles['q'] = r_prime2state[p]
        elif p in r_prime2symbol and roles['a'] is None:
            roles['a'] = r_prime2symbol[p]
        elif p in r_prime2state and roles['q2'] is None:
            # second state
            if roles['q'] is None:
                roles['q'] = r_prime2state[p]
            else:
                roles['q2'] = r_prime2state[p]
        elif p in r_prime2symbol and roles['b'] is None:
            roles['b'] = r_prime2symbol[p]
        elif p in r_prime2dir and roles['D'] is None:
            roles['D'] = r_prime2dir[p]
        else:
            # fallback: try to classify by prime ranges if ambiguous
            if p in r_prime2state and roles['q2'] is None:
                roles['q2'] = r_prime2state[p]
            elif p in r_prime2symbol and roles['b'] is None:
                roles['b'] = r_prime2symbol[p]
            elif p in r_prime2dir and roles['D'] is None:
                roles['D'] = r_prime2dir[p]

    return (roles['q'], roles['a'], roles['q2'], roles['b'], roles['D'])

# construct transition table (emap from (state,symbol) -> (state',symbol',dir))
transition_table = {}
for T in r_trans_enc:
    tup = decode_transition_integer(T)
    key = (tup[0], tup[1])
    transition_table[key] = (tup[2], tup[3], tup[4])

print("\nReceiver transition table reconstructed:")
for k,v in transition_table.items():
    print(f"  {k} -> {v}")

# --------------------------
# Decode payload configuration integer into state + tape dict
# (must match the same encoding scheme used by sender: state * product(sym_prime ** (pos+2)))
# --------------------------
def decode_configuration(conf_int, state_map=r_state2prime, sym_map=r_symbol2prime, prime_bases=PRIMES):
    fac = factorize(conf_int, prime_bases)
    # find state by matching which state-prime divides conf_int (state primes are unique)
    state = None
    for st, p in state_map.items():
        if p in fac:
            state = st
            break
    # reconstruct tape by scanning symbol primes and converting exponent to position
    tape = {}
    for p,exp in fac.items():
        if p in r_prime2symbol:
            pos = exp - 2  # inverse of encoding: exponent = pos + 2
            tape[pos] = r_prime2symbol[p]
    # normalize tape keys ascending
    tape_norm = {pos:tape[pos] for pos in sorted(tape.keys())}
    return state, tape_norm, fac

recv_conf_int = int(recv['payload_conf'])
r_state, r_tape, r_fac = decode_configuration(recv_conf_int)
print("\nReceiver decoded initial configuration:")
print(" state:", r_state)
print(" tape:", r_tape)
print(" prime factorization (partial):", r_fac)

# --------------------------
# Simulate TM using reconstructed transition table.
# We'll assume head starts at position 0 and we have infinite blank to the right (if position absent -> B)
# Execution loop: at each step: read symbol at head pos (default 'B' if missing), find transition, apply:
#   write new symbol, move head, set new state. If no transition applicable -> halt.
# --------------------------
def simulate_tm(state, tape, transition_table, blank_symbol='B', max_steps=1000):
    head = 0
    step = 0
    history = []
    while step < max_steps:
        sym = tape.get(head, blank_symbol)
        key = (state, sym)
        # record
        history.append((step, state, head, sym))
        if key not in transition_table:
            # no rule -> halt
            print(f"[step {step}] HALT: no rule for (state={state}, sym={sym})")
            break
        new_state, write_sym, direction = transition_table[key]
        # apply write
        if write_sym == 'B':
            # writing blank: remove cell if exists
            if head in tape:
                del tape[head]
        else:
            tape[head] = write_sym
        # move head
        if direction == 'R':
            head += 1
        elif direction == 'L':
            head -= 1
        else:
            pass
        # update state
        state = new_state
        # halting if we've moved into a halting state name (qH) or no transitions for it
        # but we'll loop and check the next iteration
        step += 1
    else:
        print("Max steps reached, aborting.")
    return state, tape, history

final_state, final_tape, hist = simulate_tm(r_state, r_tape.copy(), transition_table)
print("\nFinal state:", final_state)
print("Final tape (sparse):", final_tape)
print("History sample (first 10 steps):")
for h in hist[:10]:
    print(h)

# For readability, present tape contents as list from pos 0..maxpos encountered
if final_tape:
    maxpos = max(final_tape.keys())
    out_list = [final_tape.get(i,'B') for i in range(0, maxpos+1)]
else:
    out_list = []
print("\nFinal tape as list (pos 0..):", out_list)

# --------------------------
# DONE: This demonstrates a full handshake + machine reconstruction + payload decoding + execution.
# --------------------------

Sender: symbol2prime: {'0': 2, '1': 3, 'B': 5}
Sender: state2prime:  {'q0': 233, 'qH': 239}
Sender: dir2prime:    {'L': 547, 'R': 557}

Sender: encoded transitions (integers):
181433838
181433838
775441475

--- Handshake JSON (to be sent) ---
{"symbol2prime": {"0": 2, "1": 3, "B": 5}, "state2prime": {"q0": 233, "qH": 239}, "dir2prime": {"L": 547, "R": 557}, "transitions": [181433838, 181433838, 775441475]}

Sender: initial configuration integer: 1019142000000
 (this integer encodes state + tape and will be sent as payload)

--- Message bundle (handshake + payload) ready to send --- (length chars): 212


=== RECEIVER: receive JSON and reconstruct grammar & machine ===
Receiver rebuilt symbol2prime: {'0': 2, '1': 3, 'B': 5}
Receiver rebuilt state2prime:  {'q0': 233, 'qH': 239}
Receiver rebuilt dir2prime:    {'L': 547, 'R': 557}

Receiver transition table reconstructed:
  ('q0', '0') -> (None, '1', 'R')
  ('q0', 'B') -> ('qH', None, 'R')

Receiver decoded initial configuration:
 state: q0